# Encrypted DataBase Backups in SQL Server

## Master Key

Para comenzar a trabajar con encriptación de bases de datos, va a ser necesario crear una master key, esta permitirá proteger los certificados que vayamos a utilizar luego, para la realización de backups.

Creación de la MasterKey:

In [ ]:
USE master 
GO 
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Password para la Master Key'


## Certificado

Es posible tener tantos como se deseé, esto añadirá un nivel de seguridad más, pero también es posible utilizar el mismo para los backups de todas las bases de datos de la instancia.

Para el ejemplo se crearán 2 certificados, se usará uno para el backup de dos bases de datos y el otro para una tercera base. 

Si no se le da una fecha de vencimiento, asume que tiene un año de vigencia desde el momento de la fecha de creación.

In [ ]:
Use Master 
GO

CREATE CERTIFICATE CertificadoBaK1
WITH SUBJECT = 'Certificado para Backups'
,EXPIRY_DATE = '20201031'; 

CREATE CERTIFICATE CertificadoBK2 
WITH SUBJECT = 'Certificado para BackupsVence06/10/2020'
,EXPIRY_DATE = '20211006'; 


## Backup de los certificados

Esto permitirá hacer la restauración de la base de datos en una instancia que no sea la original. 

Las contraseñas pueden ser tan complejas como se crea necesario, así como es posible usar un único certificado para realizar el backup a todas las bases de la instancia, también es posible mantener la misma contraseña tanto para la _master  key_ como para el o los _certificados_. Por supuesto, _esto último no se recomienda._

Un último punto a tener en cuenta es que el agente tenga permisos sobre la carpeta que contendrá los archivos .cer y .pvk

In [ ]:
USE master 
GO 
BACKUP CERTIFICATE CertificadoBak  
TO FILE = 'Ruta\BKPCertificadoBak.cer' 
WITH PRIVATE KEY (FILE = 'Ruta\Backup_Key.pvk' , 
ENCRYPTION BY PASSWORD = 'Password para el certificado')
GO 

USE master 
GO 
BACKUP CERTIFICATE CertificadoBak2
TO FILE = 'Ruta\BKPCertificadoBak2.cer' 
WITH PRIVATE KEY (FILE = 'Ruta\Backup_Key2.pvk' , 
ENCRYPTION BY PASSWORD = 'Password para el certificado 2')
GO 

## Backups Encriptados de Bases de Datos

Ahora sí llegamos al primer punto interesante. Realizar una copia de seguridad encriptada. Algoritmos de encriptación, hay varios, pero el más recomendado es el AES\_256.

In [ ]:
BACKUP DATABASE BD1
TO DISK = N'RutaBackups\BD1Encrypt.bak' 
 WITH 
 INIT,
 ENCRYPTION 
 ( 
 ALGORITHM = AES_256, 
 SERVER CERTIFICATE = CertificadoBak 
 ), 
 COMPRESSION,
 FORMAT,
 STATS = 10 
 GO

BACKUP DATABASE BD2
TO DISK = N'RutaBackups\BD2Encrypt.bak' 
 WITH 
 INIT,
 ENCRYPTION 
 ( 
 ALGORITHM = AES_256, 
 SERVER CERTIFICATE = CertificadoBak2 
 ), 
 COMPRESSION,
 FORMAT,
 STATS = 10 
 GO

BACKUP DATABASE BD3
TO DISK = N'RutaBackups\BD3Encrypt.bak' 
 WITH 
 INIT,
 ENCRYPTION 
 ( 
 ALGORITHM = AES_256, 
 SERVER CERTIFICATE = CertificadoBak 
 ), 
 COMPRESSION,
 FORMAT,
 STATS = 10 
 GO

 El hecho de hacer un backup encriptado, no significa que no es posible hacer un backup sin encriptar:

In [ ]:
BACKUP DATABASE BD1
TO DISK = N'RutaBackups\BD1_SINEncrip.bak' 
 WITH 
 INIT,
 FORMAT,
 COMPRESSION,
 STATS = 10 
 GO

## Certificados en destino

Restuarar una base cuyo archivo de backup está encriptado, requiere que se creé una Master Key en la instancia destino que pueda manipular los Certificados que trae el bak. Si se intenta hacer la restauración sin antes crear los certificados, se abrirá una ventana indicando el error. (Pueden hacer la prueba)

Por lo tanto, primero, se crea una MasterKey (no debe tener la misma contraseña que en la otra instancia), como se indicó en el primer paso, luego, se crea el certificado correspondiente a la base de datos que se va a restaurar, pero esta vez, utilizando el archivo de backup del certificado de la instancia origen. Llevar los archivos .cer y .pkb  a la instancia destino:

In [ ]:
USE master 
GO 
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Password para la masterKey de esta otr instancia'

 CREATE CERTIFICATE CertificadoBak 
 FROM FILE = 'RutaCertificadoENDestino\CertificadoBak.cer'
 WITH PRIVATE KEY (FILE = 'RutaKeyENDestino\Backup_Key.pvk',  
 DECRYPTION BY PASSWORD = 'Password para el certificado'); 
 GO  

## Restauración De la Base de Datos

Ahora se puede proceder a restaurar las base de datos como siempre.

## Limpieza

Una vez que terminadas las tareas que requerían del certificado, se elimina de manera que no pueda volver a hacer un restore de otra copia encriptada que utilice ese certificado ni ser usado para realizar un backup con el mismo si no corresponde.

In [ ]:
DROP CERTIFICATE CertificadoBak

# Certificado Expirado

No hay problema, se elimina y se crea nuevamente con una nueva fecha. Por supuesto, se puede ser previsor y tener una tarea que realice ésto (para ello hacer uso de la vista dinámica **sys.certificates**) sin tener que estar revisado una agenda :).

# Buenas Prácticas

Mantener los backups de los certificados fuera de la instancia, como se hace con los backups de las bases de datos.

No usar la misma contraseña para las Master Key de distintas instancias, ni para los certificados.